In [5]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   To use it you need Bing maps API key. Get it here: https://www.bingmapsportal.com
   and place it into a text file named MyBingMapApi.key, where the script reads it.

   Distances are in kiolometers.
"""

#python -m pip install --upgrade --user ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from urllib.request import urlopen, Request

import json

from pprint import pprint
import numpy as np
import plotly.express as px
import pandas as pd


points   =   [['latitude', 'longitude'],
             [38.191164, 27.363971], #Warehouse location
             [40.84265, 38.14688],
             [37.26016, 28.218121269543],
             [37.05769, 27.3714988094863],
             [36.69511, 29.0718075618722],
             [36.96657, 28.2856448543648],
             [37.42706, 35.789598772996]]


def getDistanceMatrix(addressDataList, travelMode, BingMapsAPIKey):
    """Calculates distance matrix for given addresses using Bing maps API."""
    coordinateList = []
    for ad in addressDataList:
        coordinateList.append( str(ad['latitude'])+","+str(ad['longitude']) )
    coordinates = ";".join(coordinateList)

    distanceUrl = ("https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix"+
        "?origins="+coordinates+"&destinations="+coordinates+
        "&travelMode="+str(travelMode)+
        "&distanceUnit=km"+
        "&key="+str(BingMapsAPIKey))
    #print(distanceUrl)
    request = Request(distanceUrl)
    response = urlopen(request)
    result = json.loads(response.read().decode(encoding='utf-8'))
    #print(result)

    N = len(addressDataList)
    D = np.zeros( (N,N) )
    for cell in result['resourceSets'][0]['resources'][0]['results']:
        D[cell['originIndex'], cell['destinationIndex']] =  cell['travelDistance'] # could also be cell['travelDuration'] 
        
    return D


def create_data_model(coordinates):
    """Stores the data for the problem."""
    # Key for Bing API and travel mode definition
    TRAVEL_MODE = 'driving' 
    mapApiKey = open('./MyBingMapApi.key', mode='r').read().strip()
    
    # Distance matrix calculations
    keys = coordinates[0]
    locations = [dict(zip(keys, values)) for values in coordinates[1:]]
    D = getDistanceMatrix(locations, TRAVEL_MODE, mapApiKey)
    print("Distance Matrix \nD = ", end="");pprint(D)
    
    data = {}
    data['distance_matrix'] = D;
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def plots(df):
    keys = df[0]
    locations = [dict(zip(keys, values)) for values in df[1:]]
    df = pd.DataFrame(locations)
    fig = px.scatter_geo(df,lat='latitude',lon='longitude')
    fig.update_layout(title = 'Customers', title_x=0.5)
    fig.show()


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}km\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}km'.format(max_route_distance))
    
    
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(points)
    
    # Marking a locations on the map
    figures = plots(points)
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')
    
if __name__ == '__main__':
    main()

Distance Matrix 
D = array([[   0.   , 1291.798,  162.576,  212.53 ,  291.911,  221.575,
         970.315],
       [1297.34 ,    0.   , 1246.338, 1517.955, 1247.119, 1270.372,
         682.876],
       [ 162.001, 1240.068,    0.   ,  103.413,  129.657,   59.321,
         956.824],
       [ 211.36 , 1515.185,  101.639,    0.   ,  230.974,  160.638,
        1058.141],
       [ 289.494, 1226.034,  131.225,  230.906,    0.   ,  108.699,
         873.655],
       [ 213.611, 1259.902,   55.342,  155.023,  102.972,    0.   ,
         976.658],
       [ 974.484,  680.522,  937.938, 1037.619,  876.866,  961.972,
           0.   ]])


Objective: 263709
Route for vehicle 0:
 0 -> 0
Distance of the route: 0km

Route for vehicle 1:
 0 -> 0
Distance of the route: 0km

Route for vehicle 2:
 0 ->  1 -> 0
Distance of the route: 2588km

Route for vehicle 3:
 0 ->  3 ->  2 ->  5 ->  4 ->  6 -> 0
Distance of the route: 2321km

Maximum of the route distances: 2588km
